In [1]:
from resume import my_resume

In [2]:
from IPython.display import display, HTML, Markdown, clear_output, FileLink
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights
import os, shutil, re
from mark_status import mark_status
import requests

# Pulling Job Data

In [3]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)
job_skill_weights = skill_weights.get_job_skill_weights(raw_job_details)

# Generation

In [4]:
skill_text_weights_con = lambda weights, job_id: weights.loc[job_id, :].groupby(level=0).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [5]:
popular_weights = job_skill_weights.mul(jobs["excitement"]/5.0, axis="index")
popular_weights = popular_weights.groupby(level=["skill", "skill text"]).sum() / popular_weights.sum()
popular_weights_dict = popular_weights.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [6]:
base_args = {
    "should_render_all":True,
    "trim_skill_html_data":True,
    "goal_small":True,
    "skill_text_weights":popular_weights_dict
}

In [7]:
my_resume.write_html_to_file(
    stylesheet='chalkboard',
    filepath="docs/index.html",
    **base_args
)

In [8]:
my_resume.write_html_to_file(
    stylesheet='lighttheme',
    filepath="docs/resumes/index.html",
    **base_args
)

In [9]:
my_resume.export_pdf(
    "docs/resumes/full.pdf",
    alt_template_prefixes={"*":"pdf"},
    **base_args
)